In [49]:
import os
import requests
import zipfile
import csv
import time

# Liste des départements en France métropolitaine

In [7]:
departements_metropole = [
    "01", "02", "03", "04", "05", "06", "07", "08", "09", "10",
    "11", "12", "13", "14", "15", "16", "17", "18", "19", "20",
    "21", "22", "23", "24", "25", "26", "27", "28", "29", "30",
    "31", "32", "33", "34", "35", "36", "37", "38", "39", "40",
    "41", "42", "43", "44", "45", "46", "47", "48", "49", "50",
    "51", "52", "53", "54", "55", "56", "57", "58", "59", "60",
    "61", "62", "63", "64", "65", "66", "67", "68", "69", "70",
    "71", "72", "73", "74", "75", "76", "77", "78", "79", "80",
    "81", "82", "83", "84", "85", "86", "87", "88", "89", "90",
    "91", "92", "93", "94", "95"
]

# Liste des départements et régions d'outre-mer (DROM)
departements_drom = ["971", "972", "973", "974", "976"]

# Liste complète des départements
liste_departements = departements_metropole + departements_drom

annees = ["2019","2020","2021","2022","2023"]


# Liens fichiers csv, zip et excel à télécharger 

In [46]:
#liste_departements

url_inondation = "https://files.georisques.fr/di_2020/tri_2020_sig_di_{0}.zip"
url_eolien ="https://files.georisques.fr/bdeole/production/csv/eolien_dept{0}.zip"
url_inondation_par_remontee_nappes = "https://files.georisques.fr/REMNAPPES/Dept_{0}.zip"
inventaire_de_cavite = "https://www.georisques.gouv.fr/webappReport/ws/cavites/departements/{0}/fiches.csv?"
mvt_terrain = "https://www.georisques.gouv.fr/webappReport/ws/mvmt/departements/{0}/fichecommunes.csv?"
obligation_legale_debrousaillement = "https://files.georisques.fr/old_feu/{0}.zip"
zonage_inondation_2013 = "https://files.georisques.fr/di/TRI_{0}_SIG_DI.zip"
zonage_inondation_2020 = "https://files.georisques.fr/di_2020/tri_2020_sig_di_{0}.zip"
retrait_gonflement_argile = "https://files.georisques.fr/argiles/AleaRG{0}_L93.zip"
secteurs_info_sols = "https://mapsref.brgm.fr/wxs/georisques/georisques_dl?&service=wfs&version=2.0.0&request=getfeature&typename=classification&outputformat=csvtext&Filter=%3CFilter%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Ecode_dpt%3C/PropertyName%3E%3CLiteral%3E{0}%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3C/Filter%3E"# dep
secteurs_sols = "https://mapsref.brgm.fr/wxs/georisques/georisques_dl?&service=wfs&version=2.0.0&request=getfeature&typename=classification&outputformat=csvtext&Filter=%3CFilter%3E%3CPropertyIsEqualTo%3E%3CPropertyName%3Ecode_dpt%3C/PropertyName%3E%3CLiteral%3E{0}%3C/Literal%3E%3C/PropertyIsEqualTo%3E%3C/Filter%3E"
site_et_sols_pollues = "https://www.georisques.gouv.fr/webappReport/ws/infosols/export/excel?codeDepartement={0}"

# liste_annee
installation_industrielle_polluant = "https://files.georisques.fr/irep/{0}.zip"

# télécharger manuellement car un seul fichier
procedures_risque = "https://files.georisques.fr/GASPAR/gaspar.zip"

# fonction

## Note explicative : 
- la fonction téléchargement créee 2  sous- dossiers (téléchargement et extraction):
        -le dossier téléchargement : il contient tous les dossiers au format zip
        -le dossier extraction : il contien tous les dossiers dézippés (au format csv, shp etc.... selon le lien fourni) 
- teléchargement(liste_dep, url, nom_dossier) : 
        - url : on attribue à la fonction l'un des liens énumérés ci-dessus 
        -liste_dep : on attribue à la fonction la liste qui correspond au lien
        - nom_dossier : on nomme le sous-dossier dans lequel, on aimerait que le fichier soit 
              - Exemple d'utilisation 
                    - liste_dep = [ "02", "03"]  
                    - nom_dossier = "secteurs_info_sols"
                    - url = secteurs_info_sols
                    **teléchargement(liste_dep,secteurs_info_sols,nom_dossier)**
                    
NB : les documents seront enregistrés dans le dossier dans lequel se trouve le notebook


In [53]:
def teléchargement(liste_dep, url, nom_dossier):
    # proxies 
    proxies = {
    'http': "http://inetproxy:83",
    'https': "http://inetproxy:83"
    }
  
    path_ = os.getcwd()
    dossier_telechargement = os.path.join(path_, fr"telechargements\{nom_dossier}")
    os.makedirs(dossier_telechargement, exist_ok=True)

    for dep_ in liste_dep:
        try:
           
            url_ = url.format(dep_)
            dossier_extraction = os.path.join(path_, fr"extractions\{nom_dossier}\{dep_}")
            os.makedirs(dossier_extraction, exist_ok=True)
          
            nom_fichier = os.path.basename(url_)

            if "csv" in url_:
                chemin_csv_local = os.path.join(dossier_extraction, f"fichier{dep_}.csv")
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                    with open(chemin_csv_local, 'wb') as fichier:
                        fichier.write(reponse.content)
                    print(f"Fichier CSV téléchargé à : {chemin_csv_local}")
                else:
                    print(f"Échec du téléchargement du fichier CSV. Code d'état : {reponse.status_code}")
                    
            elif  "excel" in url_:
                chemin_excel_local = os.path.join(dossier_extraction, f"fichier{dep_}.xlsx")
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                    with open(chemin_excel_local, 'wb') as fichier:
                        fichier.write(reponse.content)
                    print(f"Fichier Excel téléchargé à : {chemin_excel_local}")
                else:
                    print(f"Échec du téléchargement du fichier Excel. Code d'état : {reponse.status_code}")
            else:
              
                chemin_zip_local = os.path.join(dossier_telechargement, nom_fichier)
                time.sleep(10)
                reponse = requests.get(url_,verify=False)
                if reponse.status_code == 200:
                
                    with open(chemin_zip_local, 'wb') as fichier:
                        fichier.write(reponse.content)

                    print(f"Fichier ZIP téléchargé à : {chemin_zip_local}")

                 
                    with zipfile.ZipFile(chemin_zip_local, 'r') as zip_ref:
                        zip_ref.extractall(dossier_extraction)

                    print(f"Fichiers extraits dans : {dossier_extraction}")
                else:
                    print(f"Échec du téléchargement du fichier ZIP. Code d'état : {reponse.status_code}")

        except Exception as e:
            print(f"Une erreur s'est produite : {e}")

In [54]:
# Exemple d'utilisation 
liste_dep = [ "02", "03"]  

nom_dossier = "test"
teléchargement(liste_dep,secteurs_info_sols,nom_dossier)

C:\Users\ezano\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapsref.brgm.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Fichier CSV téléchargé à : C:\Users\ezano\Documents\GitHub\alerting2\georisque\extractions\test\02\fichier02.csv


C:\Users\ezano\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mapsref.brgm.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


Fichier CSV téléchargé à : C:\Users\ezano\Documents\GitHub\alerting2\georisque\extractions\test\03\fichier03.csv
